In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

### SETTINGS ###
base_dir = os.path.join("..", "..", "..")
f_samples_sbg = os.path.join(base_dir, "2_data", "2_image_sampling", "salzburg_sample.gpkg")
f_samples_wup = os.path.join(base_dir, "2_data", "2_image_sampling", "wuppertal_sample.gpkg")
output_dir = "output"
image_dir = os.path.join(base_dir, "3_images")
f_ratings_workshop = os.path.join(base_dir, "4_results", "2_img_workshop.csv")
f_ratings_arena = os.path.join(base_dir, "4_results", "3_img_conference-arena.csv")
f_survey = os.path.join(base_dir, "4_results", "1_online_survey.csv")
# mapping of images used in the conference workshop to sample IDs
image_mapping_a = ["img_1", "img_2", "img_3", "img_4", "img_5"]
image_mapping_b = ["wuppertal1i18", "wuppertal2i23", "wuppertal3i5", "wuppertal4i25", "wuppertal5i6"]

### plot / export settings
export_png = False
export_html = False
export_svg = False
export_pdf = True
show_title = False
show_ba_lab = False
detail_plot_all = False # individual plot per image (for all or high response rate)

### library settings ###
pd.options.plotting.backend = "plotly"

### SAMPLING DATA (with bikeability index and geometry of road segments) ###
samples_sbg = gpd.read_file(f_samples_sbg)
samples_wup = gpd.read_file(f_samples_wup)
print("Salzburg:", len(samples_sbg), "samples, Wuppertal:", len(samples_wup), "samples")
samples = samples_wup.append(samples_sbg)

### BIKEABILITY IMAGE RATINGS ###
def rename_cols(input_name):
    i = image_mapping_a.index(input_name)
    if i < 0:
        raise Exception("Unknown mapping for input column " + input_name)
    return image_mapping_b[i]
ratings_workshop = pd.read_csv(f_ratings_workshop, sep=";", decimal=",").rename(columns=rename_cols)
ratings_arena = pd.read_csv(f_ratings_arena, sep=";", decimal=",").rename(columns=rename_cols)
ratings_conf_full = ratings_workshop.append(ratings_arena)
display(ratings_workshop)
display(ratings_arena)

### SURVEY ###
survey = pd.read_csv(f_survey, sep=";", decimal=",")
slen = len(survey)
# filter survey for submitted responses
survey = survey[survey.lastpage == 7]
ssublen = len(survey)
display(f"{slen} survey hits and {ssublen} submitted answers. Response rate: {ssublen/slen:.1%}")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Survey - introduction

In [ ]:
display(survey.cyclist_type.hist())
vc = survey.cyclist_type.value_counts()
print("Type of cyclist [%]")
print(vc/ssublen*100)

In [ ]:
display(survey.country.sort_values().hist())
vc = survey.country.value_counts()
print("Country of residence [%]")
print(vc/ssublen*100)

In [ ]:
display(survey.gender.hist())
vc = survey.gender.value_counts()
print("Gender [%]")
print(vc/ssublen*100)

In [ ]:
# extract sample IDs of images used in the survey
survey_cols = survey.columns.to_list()
survey_sampleIDs = []
for c in survey_cols:
    if c.startswith("salzburg") or c.startswith("wuppertal"):
        if c.endswith("Time"):
            continue
        survey_sampleIDs.append(c)
print(len(survey_sampleIDs), "samples available in survey")
samples_sel = samples.set_index("sample_id")
samples_sel = samples_sel.loc[survey_sampleIDs]
bikeability = samples_sel["index"].sort_values()
bikeability_class = (bikeability*10/2).apply(np.ceil).apply(int)
ratings_survey = survey[survey_sampleIDs]
ratings_survey

In [ ]:
s_survey = ratings_survey.describe().T
s_survey["bikeability"] = bikeability
s_survey.head()

In [ ]:
s_workshop = ratings_workshop.describe().T
s_workshop["bikeability"] = bikeability
s_workshop

In [ ]:
s_arena = ratings_arena.describe().T
s_arena["bikeability"] = bikeability
s_arena

In [ ]:

def def_max_dev(val):
    if val < 0.5:
        return 1 - val
    return val

def comp_mad_median(col):
    t_median = col.quantile(0.5)
    delta = (col - t_median).abs()
    return delta.mean()
    
def generate_rating_stats(input_data, title):
    sdf = input_data.describe().T
    # add interquartile range, interquantile range (iqr), and mean of absolute deviation (based on mean)
    sdf["10%"] = input_data.quantile(0.1)
    sdf["1/6"] = input_data.quantile(1/6)
    sdf["5/6"] = input_data.quantile(5/6)
    sdf["90%"] = input_data.quantile(0.9)
    sdf["mad"] = input_data.mad(axis=0, skipna=True) # MAD based on mean
    sdf["mad_median"] = input_data.apply(comp_mad_median) # MAD based on median
    sdf["iqr"] = sdf["75%"] - sdf["25%"]
    sdf["iquantr"] = sdf["90%"] - sdf["10%"]
    sdf["iquantr6"] = sdf["5/6"] - sdf["1/6"]
    
    # add bikeability and deviation metrics relative to bikeability
    sdf["bikeability"] = bikeability
    sdf["delta_mean"] = sdf.bikeability - sdf["mean"]
    sdf["delta_median"] = sdf.bikeability - sdf["50%"]
    sdf["z_score"] = sdf.delta_mean / sdf["std"]
    sdf = sdf.sort_values("bikeability")
    
    # add boolean indication for whether bike index is within stat. disp. range...
    sdf["in_std"] = sdf["std"] >= sdf.delta_mean.abs()
    sdf["in_2std"] = sdf["std"]*2 >= sdf.delta_mean.abs()
    sdf["in_3std"] = sdf["std"]*3 >= sdf.delta_mean.abs()
    sdf["in_mad"] = sdf["mad"] >= sdf.delta_mean.abs()
    sdf["in_mad_median"] = sdf["mad_median"] >= sdf.delta_median.abs()
    sdf["in_iqr"] = (sdf.bikeability <= sdf["75%"]) & (sdf.bikeability >= sdf["25%"])
    sdf["in_iquantr"] = (sdf.bikeability <= sdf["90%"]) & (sdf.bikeability >= sdf["10%"])
    sdf["in_iquantr6"] = (sdf.bikeability <= sdf["5/6"]) & (sdf.bikeability >= sdf["1/6"])
    
    # compute absolute difference between each rating and computed bikeability
    cols = bikeability[input_data.columns].sort_values(ascending=True).index
    abs_dif = input_data[cols] - bikeability[cols]
    
    plot = abs_dif.boxplot(points="all", title=f"Absolute differences: {title}")
    plot.update_traces(marker_color="#15459C", line_width=1, marker_size=2)
    display(plot)
    print("absolute differences")
    display(abs_dif.describe())
    
    plot = abs(abs_dif).boxplot(points="all", title=f"|Absolute differences|: {title}")
    plot.update_traces(marker_color="#15459C", line_width=1, marker_size=2)
    display(plot)
    print("|absolute differences|")
    display(abs(abs_dif).describe())
    
    n_all = len(sdf)
    n_sd05 = sum(sdf.z_score.abs()<0.5)
    n_sd1 = sum(sdf.z_score.abs()<1)
    n_sd15 = sum(sdf.z_score.abs()<1.5)
    n_sd2 = sum(sdf.z_score.abs()<2)
    print(f"""Within \t+/- 1 sd: {n_sd1}/{n_all} = {n_sd1/n_all:.1%} 
      \n\t+/- 0.5 sd: {n_sd05/n_all:.1%} 
      \t+/- 1 sd: {n_sd1/n_all:.1%}
      \t+/- 1.5 sd: {n_sd15/n_all:.1%}
      \t+/- 2 sd: {n_sd2/n_all:.1%}""")
    
    # add bikeability class columns (for bike index, median rating and mean rating)
    sdf["bikeability_class"] = (sdf.bikeability*5).apply(np.ceil).apply(int)
    sdf["median_rating_ba_class"] = (sdf["50%"]*5).apply(np.ceil).apply(int)
    sdf["mean_rating_ba_class"] = (sdf["mean"]*5).apply(np.ceil).apply(int)
    sdf.bikeability_class.replace(0,1, inplace=True)
    sdf.median_rating_ba_class.replace(0,1, inplace=True)
    sdf.mean_rating_ba_class.replace(0,1, inplace=True)
    
    # difference in bikeability class (ratings vs. NetAScore bike index)
    sdf["d_bac_median"] = sdf.bikeability_class - sdf.median_rating_ba_class
    sdf["d_bac_mean"] = sdf.bikeability_class - sdf.mean_rating_ba_class
    
    # additional classes: binary and 3 classes of bikeability
    sdf["bac_2_bikeability"] = (sdf.bikeability*2).apply(np.ceil).apply(int)
    sdf["bac_2_median_rating"] = (sdf["50%"]*2).apply(np.ceil).apply(int)
    sdf["bac_2_mean_rating"] = (sdf["mean"]*2).apply(np.ceil).apply(int)
    sdf.bac_2_bikeability.replace(0,1, inplace=True)
    sdf.bac_2_median_rating.replace(0,1, inplace=True)
    sdf.bac_2_mean_rating.replace(0,1, inplace=True)
    sdf["d_bac_2_median"] = sdf.bac_2_bikeability - sdf.bac_2_median_rating
    sdf["d_bac_2_mean"] = sdf.bac_2_bikeability - sdf.bac_2_mean_rating
    
    sdf["bac_3_bikeability"] = (sdf.bikeability*3).apply(np.ceil).apply(int)
    sdf["bac_3_median_rating"] = (sdf["50%"]*3).apply(np.ceil).apply(int)
    sdf["bac_3_mean_rating"] = (sdf["mean"]*3).apply(np.ceil).apply(int)
    sdf.bac_3_bikeability.replace(0,1, inplace=True)
    sdf.bac_3_median_rating.replace(0,1, inplace=True)
    sdf.bac_3_mean_rating.replace(0,1, inplace=True)
    sdf["d_bac_3_median"] = sdf.bac_3_bikeability - sdf.bac_3_median_rating
    sdf["d_bac_3_mean"] = sdf.bac_3_bikeability - sdf.bac_3_mean_rating
    
    #display(sdf.d_bac_median.hist())
    #display(sdf.d_bac_2_median.hist())
    #display(sdf.d_bac_3_median.hist())
    
    # summary for bikeability classes - here only considering median
    def summarize_bac_stats(bac_diff):
        matched = 0
        max_1 = 0
        less = 0
        more = 0
        greater_1 = 0
        total = 0
        for k, v in bac_diff.value_counts().items():
            total += v
            if k < 0:
                less += v
            elif k > 0:
                more += v
            else:
                matched += v
            if abs(k) > 1:
                greater_1 += v
            elif abs(k) > 0:
                max_1 += v    
        print(f"Out of {total} in total, {matched} are matched; {max_1} within +/- 1; {greater_1} outside +/- 1; {less} below and {more} higher than rated")
        print(f"Out of {total} in total, {matched/total:.1%} are matched; {max_1/total:.1%} within +/- 1; {greater_1/total:.1%} outside +/- 1; {less/total:.1%} below and {more/total:.1%} higher than rated")
    
    print("Differences in bikeability classes:")
    print(sdf.d_bac_median.value_counts())
    summarize_bac_stats(sdf.d_bac_median)
    print("...for binary classification (bikeable / unbikeable):")
    summarize_bac_stats(sdf.d_bac_2_median)
    print("...and for 3-class classification (bikeable / medium / unbikeable):")
    summarize_bac_stats(sdf.d_bac_3_median)
    
    # plot statistical dispersion metrics
    plot = sdf[["iqr", "iquantr", "iquantr6", "mad", "mad_median", "std"]].boxplot(points="all", title=f"Statistical dispersion: {title}")
    plot.update_traces(marker_color="#15459C", line_width=1, marker_size=2)
    display(plot)
    
    
    # show summary
    print(f"Summary statistics for {title}")
    display(sdf)
    
    # summarize across images/samples
    n_sample = len(sdf)
    print(f"\nmedian of bikeability ratings:")
    print(sdf["50%"].describe())
    print("Share of samples (images) per bikeability class (based on median rating) [%]:")
    print(sdf.median_rating_ba_class.value_counts()/n_sample*100)
    # statistical dispersion
    def summarize_dispersion(data, title):
        print(f"\nStatistical dispersion of image ratings ({title}):")
        print(f"stdev: {data['std'].mean():.2f}")
        print(f"MAD (mean-based): {data['mad'].mean():.2f}")
        print(f"MAD (median-based): {data['mad_median'].mean():.2f}")
        print(f"IQR (0.25; 0.75): {data['iqr'].mean():.2f}")
        print(f"Interdecile range (0.1; 0.9): {data['iquantr'].mean():.2f}")

    summarize_dispersion(sdf, "full sample")
    summarize_dispersion(sdf[sdf.bikeability_class==1], "bikeability class 1")
    summarize_dispersion(sdf[sdf.bikeability_class==2], "bikeability class 2")
    summarize_dispersion(sdf[sdf.bikeability_class==3], "bikeability class 3")
    summarize_dispersion(sdf[sdf.bikeability_class==4], "bikeability class 4")
    summarize_dispersion(sdf[sdf.bikeability_class==5], "bikeability class 5")
    
    
    # advanced summary
    print(f"\nSummary statistics across all {n_sample} images/samples in relation to bike index for: {title}")
    print(f"Mean Absolute Error (median rating vs. modeled bikeability): {sdf.delta_median.abs().sum()/len(sdf):.2f}")
    print(f"RMSE (median rating vs. modeled bikeability): {(sdf.delta_median ** 2).mean() ** 0.5 :.2f}\n")
    print(f"z-score: {sdf.z_score.describe()}")
    c_in_std = sdf.in_std.sum() / n_sample
    c_in_2std = sdf.in_2std.sum() / n_sample
    c_in_3std = sdf.in_3std.sum() / n_sample
    c_in_mad = sdf.in_mad.sum() / n_sample
    c_in_mad_median = sdf.in_mad_median.sum() / n_sample
    c_in_iqr = sdf.in_iqr.sum() / n_sample
    c_in_iquantr = sdf.in_iquantr.sum() / n_sample
    c_in_iquantr6 = sdf.in_iquantr6.sum() / n_sample
    print(f"Share of images/samples within...\n  stdev:\t{c_in_std:.1%}\n  2 x stdev:\t{c_in_2std:.1%}\n  3 x stdev:\t{c_in_3std:.1%}\n  MAD (mean):\t{c_in_mad:.1%}\n \tMAD (median):\t{c_in_mad_median:.1%}")
    print(f"  IQR:\t{c_in_iqr:.1%}\n  Inter quantile range (.1;.9):\t{c_in_iquantr:.1%}\n  Inter quantile range (1/6;5/6):\t{c_in_iquantr6:.1%}")
    
    return sdf

sdf = generate_rating_stats(ratings_conf_full, title="conference (workshop and arena)")

In [ ]:
# plotting function
import PIL as pil
def plot_ws_img_results(data, title, img_ids):
    plot = data[img_ids].boxplot(title=title if show_title else None, points="all")
    plot.update_layout(
        autosize=False,
        width=700,
        height=500,
        margin=dict(t=55 if show_title else 15, l=80 if show_ba_lab else 10, r=30 if show_ba_lab else 10, b=120 if show_title else 110),
        yaxis=dict(title="bikeability" if show_ba_lab else None),
        xaxis=dict(title="")
    )
    plot.update_traces(marker_color="#15459C", line_width=1, marker_size=2)
    for imgID in img_ids:
        plot.add_layout_image(
            dict(
                source=pil.Image.open(f"{image_dir}/{imgID}.jpg"),
                x=imgID,
                xref="x",
                yref="paper",
                y=-0.08 if show_title else -0.06,
                sizex=0.8,
                sizey=0.25,
                sizing="contain",
                xanchor="center",
                yanchor="top"
            )
        )
    display(plot.add_scatter(name="bikeability", y=bikeability[img_ids], x=img_ids, marker_size=60, line_color="#EB0F09", showlegend=False, mode="markers", marker_symbol="line-ew-open"))
    if export_html: plot.write_html(os.path.join(output_dir, f"{title}.html"))
    if export_png: plot.write_image(os.path.join(output_dir, f"{title}.png"), scale=4)
    if export_svg: plot.write_image(os.path.join(output_dir, f"{title}.svg"))
    if export_pdf: plot.write_image(os.path.join(output_dir, f"{title}.pdf"))

In [ ]:
plot_ws_img_results(ratings_workshop, "Bikeability ratings: conference workshop", image_mapping_b)

In [ ]:
plot_ws_img_results(ratings_arena, "Bikeability ratings: conference arena", image_mapping_b)

In [ ]:
plot_ws_img_results(ratings_conf_full, "Bikeability ratings: conference (workshop and arena)", image_mapping_b)

In [ ]:
plot_ws_img_results(ratings_survey[image_mapping_b]/100, "Bikeability ratings: survey", image_mapping_b)

In [ ]:
survey_topResponseRate = ratings_survey.count().sort_values(ascending=False)
s_top5 = survey_topResponseRate.head(5).index
s_top5 = bikeability[s_top5].sort_values().index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - Top 5 response rate", s_top5)

In [ ]:
s_next5 = survey_topResponseRate[5:10].index
s_next5 = bikeability[s_next5].sort_values().index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - Next 5 (6-10) by response rate", s_next5)

In [ ]:
s_next5b = survey_topResponseRate[10:15].index
s_next5b = bikeability[s_next5b].sort_values().index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - Next 5 (11-15) by response rate", s_next5b)

In [ ]:
s_next5c = survey_topResponseRate[15:20].index
s_next5c = bikeability[s_next5c].sort_values().index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - Next 5 (16-20) by response rate", s_next5c)

In [ ]:
# TOP 5 by bikeabilty class
s_bc1 = survey_topResponseRate[bikeability_class[bikeability_class == 1].index].sort_values(ascending=False).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - bikeability class 1 - Top response rate", s_bc1)

In [ ]:
s_bc2 = survey_topResponseRate[bikeability_class[bikeability_class == 2].index].sort_values(ascending=False).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - bikeability class 2 - Top response rate", s_bc2)

In [ ]:
s_bc3 = survey_topResponseRate[bikeability_class[bikeability_class == 3].index].sort_values(ascending=False).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - bikeability class 3 - Top response rate", s_bc3)

In [ ]:
s_bc4 = survey_topResponseRate[bikeability_class[bikeability_class == 4].index].sort_values(ascending=False).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - bikeability class 4 - Top response rate", s_bc4)

In [ ]:
s_bc5 = survey_topResponseRate[bikeability_class[bikeability_class == 5].index].sort_values(ascending=False).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - bikeability class 5 - Top response rate", s_bc5)

In [ ]:
sdf_survey_full = generate_rating_stats(ratings_survey/100, title="survey")

In [ ]:
# by response variability
i_rv_l = sdf_survey_full["iqr"].sort_values().head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - lowest response variability", i_rv_l)

In [ ]:
i_rv_h = sdf_survey_full["iqr"].sort_values(ascending=False).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - highest response variability", i_rv_h)


In [ ]:
# 5 top matched (estimated vs. median rating; based on z-score)
i_zscore_l = sdf_survey_full.z_score.abs().sort_values().head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - lowest absolute z-score", i_zscore_l)

In [ ]:
# 5 most disagreed (estimated vs. median rating; based on z-score)
i_zscore_h = sdf_survey_full.z_score.abs().sort_values(ascending=False).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - highest absolute z-score", i_zscore_h)

In [ ]:
# 5 best fit (estimated vs. median rating; based on absolute deviation)
i_absdev_l = sdf_survey_full.delta_median.abs().sort_values(ascending=True).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - lowest absolute deviation", i_absdev_l)

In [ ]:
# 5 most disagreed (estimated vs. median rating; based on absolute deviation)
i_absdev_h = sdf_survey_full.delta_median.abs().sort_values(ascending=False).head(5).index
plot_ws_img_results(ratings_survey/100, "Bikeability ratings: survey - highest absolute deviation", i_absdev_h)

In [ ]:
# order columns by bikeability index (ascending, was sorted beforehand)
plot = (ratings_survey / 100).reindex(bikeability.index, axis=1).boxplot(title="Survey ratings (all samples)" if show_title else None)
plot.update_layout(
    width=1200, height=500,
    xaxis=dict(title=""), yaxis=dict(title="bikeability" if show_ba_lab else None),
    margin=dict(t=55 if show_title else 15, l=80 if show_ba_lab else 10, r=30 if show_ba_lab else 10, b=120 if show_title else 110)
)
plot.update_traces(marker_color="#15459C", line_width=1, marker_size=2)
display(plot.add_scatter(name="bikeability", y=bikeability, x=bikeability.index, marker_size=10, line_color="#EB0F09", showlegend=False, mode="markers", marker_symbol="line-ew-open"))
if export_html: plot.write_html(os.path.join(output_dir, "survey_ratings_all.html"))
if export_png: plot.write_image(os.path.join(output_dir, "survey_ratings_all.png"), scale=6)
if export_svg: plot.write_image(os.path.join(output_dir, "survey_ratings_all.svg"))
if export_pdf: plot.write_image(os.path.join(output_dir, "survey_ratings_all.pdf"))

In [ ]:
surveyIDs_high_response = s_survey[s_survey["count"] > 50].index
surveyIDs_high_response

In [ ]:
ti = []
for ind in bikeability.index:
    if ind in surveyIDs_high_response:
        ti.append(ind)
plot = (ratings_survey[surveyIDs_high_response].reindex(ti, axis=1)/100).boxplot(points="all", title="Survey ratings (high participation rate)" if show_title else None)
plot.update_layout(
    width=1200, height=600,
    xaxis=dict(title=""), yaxis=dict(title=""), 
    margin=dict(t=55 if show_title else 15, l=80 if show_ba_lab else 10, r=30 if show_ba_lab else 10, b=110 if show_title else 100)
)
plot.update_traces(marker_color="#15459C", line_width=1, marker_size=1.5)
display(plot.add_scatter(name="bikeability", y=bikeability[ti], x=ti, marker_size=25, line_color="#EB0F09", showlegend=False, mode="markers", marker_symbol="line-ew-open"))
if export_html: plot.write_html(os.path.join(output_dir, "survey_ratings_highParticipation.html"))
if export_png: plot.write_image(os.path.join(output_dir, "survey_ratings_highParticipation.png"), scale=6)
if export_svg: plot.write_image(os.path.join(output_dir, "survey_ratings_highParticipation.svg"))
if export_pdf: plot.write_image(os.path.join(output_dir, "survey_ratings_highParticipation.pdf"))

In [ ]:
# correlation (using median of ratings)
survey_median = ratings_survey.agg("median", 0)
s_corr_p = bikeability.corr(survey_median/100) # pearson
s_corr_s = bikeability.corr(survey_median/100, method="spearman") # spearman 
s_corr_k = bikeability.corr(survey_median/100, method="kendall") # kendall 
print(f"Correlations (full survey):\n{s_corr_p:.3f} Pearson\n{s_corr_s:.3f} Spearman\n{s_corr_k:.3f} Kendall\n")

survey_hr_median = ratings_survey[surveyIDs_high_response].agg("median", 0)
s_hr_corr_p = bikeability.corr(survey_hr_median/100) # Pearson's r
s_hr_corr_s = bikeability.corr(survey_hr_median/100, method="spearman") # Spearman's rho
s_hr_corr_k = bikeability.corr(survey_hr_median/100, method="kendall") # Kendall's tau
print(f"Correlations (survey - high response rate):\n{s_hr_corr_p:.3f} Pearson\n{s_hr_corr_s:.3f} Spearman\n{s_hr_corr_k:.3f} Kendall\n")

conf_median = ratings_conf_full.agg("median", 0)
conf_corr_p = bikeability.corr(conf_median) # Pearson's r
conf_corr_s = bikeability.corr(conf_median, method="spearman") # Spearman's rho
conf_corr_k = bikeability.corr(conf_median, method="kendall") # Kendall's tau
print(f"Correlations (full conference):\n{conf_corr_p:.3f} Pearson\n{conf_corr_s:.3f} Spearman\n{conf_corr_k:.3f} Kendall")

In [ ]:
import scipy.stats
tmp_comb = pd.DataFrame({"a":bikeability, "b":(survey_median/100)})
cor_result = scipy.stats.linregress(tmp_comb.b, tmp_comb.a)
cor_result

In [ ]:
plot = px.scatter(sdf_survey_full, x="50%", y="bikeability", labels={"50%":"median image-based rating",
                                                                     "bikeability": "modelled bikeability"})
plot.update_layout(
    margin=dict(t=55 if show_title else 15, l=80 if show_ba_lab else 10, r=30 if show_ba_lab else 10, b=60 if show_title else 55)
)
plot.update_traces(
        marker_color="#15459C", line_width=0, marker_size=5
    )
plot.add_trace(go.Line(x=[0,1], y=[0,1], mode="lines", name="optimum", line=dict(color="#888888", width=1, dash="dot")))
plot.add_trace(go.Line(x=[0,1], y=[cor_result.intercept,cor_result.intercept+cor_result.slope], mode="lines", name="best fit", line=dict(color="#069283", width=1)))
plot.show()
if export_html: plot.write_html(os.path.join(output_dir, "survey_ratings_all_median_scatter.html"))
if export_png: plot.write_image(os.path.join(output_dir, "survey_ratings_all_median_scatter.png"), scale=6)
if export_svg: plot.write_image(os.path.join(output_dir, "survey_ratings_all_median_scatter.svg"))
if export_pdf: plot.write_image(os.path.join(output_dir, "survey_ratings_all_median_scatter.pdf"))

In [ ]:
sdf_survey_full["residuals"] = sdf_survey_full.bikeability - (sdf_survey_full["50%"] * cor_result.slope + cor_result.intercept)
# residual plot
plot = px.scatter(sdf_survey_full, x="50%", y="residuals", labels={"50%":"median image-based rating",
                                                                     "residuals": "residuals (linear best fit)"})
plot.update_layout(
    margin=dict(t=55 if show_title else 15, l=80 if show_ba_lab else 10, r=30 if show_ba_lab else 10, b=60 if show_title else 55)
)
plot.update_traces(
    marker_color="#15459C", line_width=0, marker_size=5
)
plot.show()
if export_html: plot.write_html(os.path.join(output_dir, "survey_ratings_all_median_residuals.html"))
if export_png: plot.write_image(os.path.join(output_dir, "survey_ratings_all_median_residuals.png"), scale=6)
if export_svg: plot.write_image(os.path.join(output_dir, "survey_ratings_all_median_residuals.svg"))
if export_pdf: plot.write_image(os.path.join(output_dir, "survey_ratings_all_median_residuals.pdf"))

In [ ]:
# plot absolute differences to median
plot = px.scatter(sdf_survey_full, x="50%", y="delta_median", labels={"50%":"median image-based rating",
                                                                     "residuals": "residuals (linear best fit)"})
plot.update_layout(
    margin=dict(t=55 if show_title else 15, l=80 if show_ba_lab else 10, r=30 if show_ba_lab else 10, b=60 if show_title else 55)
)
plot.update_traces(
    marker_color="#15459C", line_width=0, marker_size=5
)
plot.show()
if export_html: plot.write_html(os.path.join(output_dir, "survey_ratings_all_delta_median.html"))
if export_png: plot.write_image(os.path.join(output_dir, "survey_ratings_all_delta_median.png"), scale=6)
if export_svg: plot.write_image(os.path.join(output_dir, "survey_ratings_all_delta_median.svg"))
if export_pdf: plot.write_image(os.path.join(output_dir, "survey_ratings_all_delta_median.pdf"))

In [ ]:
# correlation (using mean instead of median)
survey_mean = ratings_survey.agg("mean", 0)
s_corr2_p = bikeability.corr(survey_mean) # pearson
s_corr2_s = bikeability.corr(survey_mean, method="spearman") # spearman 
s_corr2_k = bikeability.corr(survey_mean, method="kendall") # kendall 
print(f"Correlations (full survey) - using mean:\n{s_corr_p:.3f} Pearson\n{s_corr_s:.3f} Spearman\n{s_corr_k:.3f} Kendall\n")

survey_hr_mean = ratings_survey[surveyIDs_high_response].agg("mean", 0)
s_hr_corr2_p = bikeability.corr(survey_hr_mean) # Pearson's r
s_hr_corr2_s = bikeability.corr(survey_hr_mean, method="spearman") # Spearman's rho
s_hr_corr2_k = bikeability.corr(survey_hr_mean, method="kendall") # Kendall's tau
print(f"Correlations (survey - high response rate) - using mean:\n{s_hr_corr_p:.3f} Pearson\n{s_hr_corr_s:.3f} Spearman\n{s_hr_corr_k:.3f} Kendall\n")

conf_mean = ratings_conf_full.agg("mean", 0)
conf_corr2_p = bikeability.corr(conf_mean) # Pearson's r
conf_corr2_s = bikeability.corr(conf_mean, method="spearman") # Spearman's rho
conf_corr2_k = bikeability.corr(conf_mean, method="kendall") # Kendall's tau
print(f"Correlations (full conference) - using mean:\n{conf_corr2_p:.3f} Pearson\n{conf_corr2_s:.3f} Spearman\n{conf_corr2_k:.3f} Kendall")

In [ ]:
# detailed plots: boxplot with modelled bikeability for each image
for id in ratings_survey.columns if detail_plot_all else ti:
    plot = ratings_survey[id].plot.box(title=f"Survey rating of {id}", orientation='h', points="all", height=250, range_x=[-1,101])
    plot.update_layout(
        xaxis=dict(title="bikeability"),
        yaxis=dict(
            visible=False
        ),
        margin=dict(t=90,l=200,r=30,b=30),
        width=850,
        height=250
    )
    plot.update_traces(marker_color="#15459C", line_width=1, marker_size=3)
    plot.add_layout_image(
        dict(
            source=pil.Image.open(os.path.join(image_dir, f"{id}.jpg")),
            y=id,
            xref="paper",
            yref="y",
            x=-0.01,
            sizex=1,
            sizey=1,
            sizing="contain", # fill / stretch / contain
            xanchor="right",
            yanchor="middle"
        )
    )
    display(plot.add_scatter(name="bikeability", x=[bikeability.loc[id]*100], y=[id], marker_size=45, line_width=12, line_color="#EB0F09", showlegend=False, mode="markers", marker_symbol="line-ns-open"))
    if export_png: plot.write_image(os.path.join(output_dir, f"survey_rating_detail_{id}.png"), scale=3)
    if export_pdf: plot.write_image(os.path.join(output_dir, f"survey_rating_detail_{id}.pdf"), scale=3)

## Qualitative part of survey

In [ ]:
survey["positive_aspects"].describe()

In [ ]:
survey[survey["positive_aspects"].isna() == False]['positive_aspects'].values

In [ ]:
survey["negative_aspects"].describe()

In [ ]:
survey[survey["negative_aspects"].isna() == False]['negative_aspects'].values

In [ ]:
survey["issues_image"].describe()

In [ ]:
survey[survey["issues_image"].isna() == False]['issues_image'].values

In [ ]:
survey["other_comments"].describe()

In [ ]:
survey[survey["other_comments"].isna() == False]['other_comments'].values

In [ ]:
f_combined = survey[(survey["positive_aspects"].isna() == False) | (survey["negative_aspects"].isna() == False) | 
       (survey["issues_image"].isna() == False) | (survey["other_comments"].isna() == False)][
              ['id', 'positive_aspects', 'negative_aspects', 'issues_image', 'other_comments']
              ]
f_combined.to_csv(os.path.join(output_dir, "feedback_combined.csv"))